# Clean Data

**INPUT**: NO INPUT

**OUTPUT**: "./data/0cleanDatasetWithQualifiersWith2025.csv"

This notebook will clean and randomize the data extracted from https://github.com/JeffSackmann/tennis_atp into something we can work with. Importantly, I want to minize data leakage (since last time it was a big concern).

## Merge Data and Basic Clean

First of all, we need some libraries.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

Let's first visualize the data a bit.

In [2]:
df = pd.read_csv("./data/all/atp_matches_2022.csv")
df.head(100)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2022-8888,Atp Cup,Hard,16,A,20220103,300,200000,NaN,NaN,Felix Auger Aliassime,R,193.0,CAN,21.4,105138,NaN,NaN,Roberto Bautista Agut,R,183.0,ESP,33.7,7-6(3) 6-3,3,F,129.0,15.0,6.0,78.0,51.0,38.0,14.0,11.0,10.0,11.0,0.0,2.0,70.0,50.0,32.0,7.0,10.0,3.0,5.0,11.0,3308.0,19.0,2260.0
1,2022-8888,Atp Cup,Hard,16,A,20220103,299,133430,NaN,NaN,Denis Shapovalov,L,185.0,CAN,22.7,105807,NaN,NaN,Pablo Carreno Busta,R,188.0,ESP,30.4,6-4 6-3,3,F,98.0,7.0,2.0,78.0,49.0,34.0,16.0,10.0,8.0,9.0,1.0,0.0,50.0,33.0,21.0,8.0,9.0,3.0,6.0,14.0,2475.0,20.0,2230.0
2,2022-8888,Atp Cup,Hard,16,A,20220103,298,105138,NaN,NaN,Roberto Bautista Agut,R,183.0,ESP,33.7,128034,NaN,NaN,Hubert Hurkacz,R,196.0,POL,24.8,7-6(6) 2-6 7-6(5),3,SF,164.0,1.0,2.0,96.0,64.0,50.0,20.0,16.0,1.0,4.0,24.0,3.0,120.0,80.0,62.0,20.0,16.0,6.0,7.0,19.0,2260.0,9.0,3706.0
3,2022-8888,Atp Cup,Hard,16,A,20220103,297,105807,NaN,NaN,Pablo Carreno Busta,R,188.0,ESP,30.4,126591,NaN,NaN,Jan Zielinski,R,180.0,POL,25.1,6-2 6-1,3,SF,53.0,6.0,0.0,45.0,33.0,25.0,8.0,8.0,0.0,0.0,2.0,1.0,38.0,27.0,17.0,1.0,7.0,4.0,8.0,20.0,2230.0,860.0,18.0
4,2022-8888,Atp Cup,Hard,16,A,20220103,296,106421,NaN,NaN,Daniil Medvedev,R,198.0,RUS,25.8,200000,NaN,NaN,Felix Auger Aliassime,R,193.0,CAN,21.4,6-4 6-0,3,SF,68.0,6.0,4.0,41.0,25.0,22.0,10.0,8.0,0.0,0.0,6.0,2.0,48.0,35.0,22.0,4.0,8.0,3.0,7.0,2.0,8640.0,11.0,3308.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2022-9665,Melbourne,Hard,32,A,20220103,286,202195,NaN,Q,Maxime Cressy,R,201.0,USA,24.6,124187,2.0,NaN,Reilly Opelka,R,211.0,USA,24.3,4-6 7-6(5) 7-6(9),3,R16,161.0,17.0,10.0,125.0,85.0,68.0,20.0,17.0,9.0,10.0,29.0,4.0,106.0,69.0,59.0,24.0,17.0,2.0,2.0,112.0,683.0,26.0,1936.0
96,2022-9665,Melbourne,Hard,32,A,20220103,284,105575,NaN,Q,Ricardas Berankis,R,175.0,LTU,31.5,106218,NaN,NaN,Marcos Giron,R,180.0,USA,28.4,7-5 6-4,3,R32,90.0,1.0,1.0,67.0,41.0,31.0,13.0,11.0,2.0,4.0,5.0,1.0,54.0,29.0,24.0,10.0,11.0,0.0,4.0,104.0,739.0,66.0,929.0
97,2022-9665,Melbourne,Hard,32,A,20220103,283,200615,NaN,NaN,Alexei Popyrin,R,196.0,AUS,22.4,105882,NaN,NaN,Stefano Travaglia,R,185.0,ITA,30.0,7-6(5) 6-3,3,R32,103.0,11.0,5.0,79.0,46.0,35.0,16.0,10.0,5.0,6.0,1.0,3.0,77.0,42.0,28.0,18.0,11.0,4.0,7.0,61.0,1000.0,78.0,846.0
98,2022-9665,Melbourne,Hard,32,A,20220103,282,134868,NaN,NaN,Tallon Griekspoor,R,188.0,NED,25.5,136440,7.0,NaN,Dominik Koepfer,L,180.0,GER,27.6,6-3 6-4,3,R32,84.0,4.0,2.0,73.0,38.0,30.0,16.0,10.0,7.0,8.0,1.0,1.0,53.0,36.0,25.0,8.0,9.0,3.0,6.0,65.0,966.0,54.0,1101.0


Let's concat all the files into one dataframe so it's easier to work with.

In [3]:
all_data = pd.read_csv("./data/all/atp_matches_1991.csv")

for year in range(1992, 2026):
    if year >= 2010:
        qual_data = pd.read_csv(f"./data/qual_chall/atp_matches_qual_chall_{year}.csv")
        all_data = pd.concat([all_data, qual_data], axis=0)
    
    file = "./data/all/atp_matches_"+str(year)+".csv"
    
    year_data = pd.read_csv(file)
    all_data = pd.concat([all_data, year_data], axis=0)

all_data

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1991-339,Adelaide,Hard,32.0,A,19901231,1,101723,NaN,NaN,Magnus Larsson,R,193.0,SWE,20.7,101414,1.0,NaN,Boris Becker,R,190.0,GER,23.1,6-4 3-6 7-6(2),3,R32,130.0,6.0,2.0,96.0,55.0,39.0,25.0,15.0,2.0,4.0,8.0,3.0,95.0,62.0,44.0,23.0,16.0,6.0,8.0,56.0,NaN,2.0,NaN
1,1991-339,Adelaide,Hard,32.0,A,19901231,2,100946,NaN,Q,Slobodan Zivojinovic,R,198.0,YUG,27.4,101256,NaN,NaN,Mark Kratzmann,L,178.0,AUS,24.6,6-3 3-6 7-6(6),3,R32,119.0,19.0,4.0,101.0,56.0,45.0,25.0,15.0,9.0,10.0,8.0,2.0,84.0,41.0,35.0,27.0,15.0,1.0,2.0,304.0,NaN,75.0,NaN
2,1991-339,Adelaide,Hard,32.0,A,19901231,3,101234,NaN,NaN,Patrik Kuhnen,R,190.0,GER,24.8,101421,NaN,NaN,Veli Paloheimo,R,183.0,FIN,23.0,6-0 6-4,3,R32,71.0,6.0,1.0,54.0,31.0,24.0,13.0,8.0,1.0,1.0,2.0,2.0,60.0,37.0,22.0,6.0,8.0,4.0,8.0,82.0,NaN,69.0,NaN
3,1991-339,Adelaide,Hard,32.0,A,19901231,4,101889,8.0,NaN,Todd Woodbridge,R,178.0,AUS,19.7,101703,NaN,NaN,Guillaume Raoux,R,180.0,FRA,20.8,7-6(2) 6-1,3,R32,85.0,2.0,0.0,60.0,40.0,30.0,14.0,9.0,3.0,3.0,3.0,3.0,74.0,45.0,30.0,11.0,10.0,5.0,8.0,50.0,NaN,84.0,NaN
4,1991-339,Adelaide,Hard,32.0,A,19901231,5,101274,NaN,NaN,Udo Riglewski,R,185.0,GER,24.4,101843,4.0,NaN,Sergi Bruguera,R,188.0,ESP,19.9,7-5 6-3,3,R32,90.0,4.0,2.0,72.0,40.0,33.0,14.0,10.0,7.0,8.0,2.0,2.0,77.0,41.0,28.0,15.0,11.0,4.0,8.0,88.0,NaN,28.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2211,2025-06-23_Eastbourne_2025-0741-381,Eastbourne,Grass,32.0,A,20250623,2025-0741-381,200221,NaN,NaN,Alejandro Davidovich Fokina,R,183.0,ESP,26.1,210150,NaN,NaN,Jakub Mensik,R,193.0,CZE,19.8,6-4 7-5,3,QF,104.0,1.0,4.0,76.0,46.0,33.0,17.0,11.0,6.0,7.0,9.0,4.0,85.0,55.0,35.0,14.0,11.0,7.0,10.0,28.0,NaN,17.0,NaN
2212,2025-06-23_Eastbourne_2025-0741-382,Eastbourne,Grass,32.0,A,20250623,2025-0741-382,126203,NaN,NaN,Taylor Fritz,R,196.0,USA,27.7,106218,NaN,NaN,Marcos Giron,R,180.0,USA,31.9,7-5 4-6 7-5,3,QF,144.0,16.0,1.0,99.0,72.0,58.0,10.0,17.0,1.0,4.0,3.0,5.0,102.0,56.0,42.0,22.0,17.0,1.0,5.0,5.0,NaN,46.0,NaN
2213,2025-06-23_Eastbourne_2025-0741-383,Eastbourne,Grass,32.0,A,20250623,2025-0741-383,202385,NaN,NaN,Jenson Brooksby,R,188.0,USA,24.7,200005,NaN,NaN,Ugo Humbert,L,188.0,FRA,27.0,6-7(7) 6-4 6-4,3,SF,167.0,4.0,6.0,104.0,56.0,39.0,26.0,16.0,9.0,13.0,4.0,3.0,103.0,65.0,44.0,16.0,16.0,7.0,13.0,149.0,NaN,20.0,NaN
2214,2025-06-23_Eastbourne_2025-0741-384,Eastbourne,Grass,32.0,A,20250623,2025-0741-384,126203,NaN,NaN,Taylor Fritz,R,196.0,USA,27.7,200221,NaN,NaN,Alejandro Davidovich Fokina,R,183.0,ESP,26.1,6-3 3-6 6-1,3,SF,93.0,18.0,0.0,77.0,54.0,42.0,12.0,13.0,2.0,3.0,1.0,2.0,65.0,43.0,30.0,11.0,12.0,0.0,3.0,5.0,NaN,28.0,NaN


As you can see, we have a lot of NaN data. We need to get rid of some that data, since it's not really useful.

Let's drop all those rows with NaN values:

In [4]:
all_data_filtered = all_data.dropna(subset=[
        'winner_id', 'loser_id', 'winner_ht', 'loser_ht', 'winner_age', 'loser_age',
        "w_ace", "w_df", "w_svpt", "w_1stIn", "w_1stWon", "w_2ndWon", "w_SvGms", "w_bpSaved", "w_bpFaced",
        "l_ace", "l_df", "l_svpt", "l_1stIn", "l_1stWon", "l_2ndWon", "l_SvGms", "l_bpSaved", "l_bpFaced",
        'winner_rank', 'loser_rank', "surface", "round", "best_of"
    ]
)
all_data_filtered = all_data_filtered.reset_index(drop=True)
all_data_filtered

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1991-339,Adelaide,Hard,32.0,A,19901231,1,101723,NaN,NaN,Magnus Larsson,R,193.0,SWE,20.7,101414,1.0,NaN,Boris Becker,R,190.0,GER,23.1,6-4 3-6 7-6(2),3,R32,130.0,6.0,2.0,96.0,55.0,39.0,25.0,15.0,2.0,4.0,8.0,3.0,95.0,62.0,44.0,23.0,16.0,6.0,8.0,56.0,NaN,2.0,NaN
1,1991-339,Adelaide,Hard,32.0,A,19901231,2,100946,NaN,Q,Slobodan Zivojinovic,R,198.0,YUG,27.4,101256,NaN,NaN,Mark Kratzmann,L,178.0,AUS,24.6,6-3 3-6 7-6(6),3,R32,119.0,19.0,4.0,101.0,56.0,45.0,25.0,15.0,9.0,10.0,8.0,2.0,84.0,41.0,35.0,27.0,15.0,1.0,2.0,304.0,NaN,75.0,NaN
2,1991-339,Adelaide,Hard,32.0,A,19901231,3,101234,NaN,NaN,Patrik Kuhnen,R,190.0,GER,24.8,101421,NaN,NaN,Veli Paloheimo,R,183.0,FIN,23.0,6-0 6-4,3,R32,71.0,6.0,1.0,54.0,31.0,24.0,13.0,8.0,1.0,1.0,2.0,2.0,60.0,37.0,22.0,6.0,8.0,4.0,8.0,82.0,NaN,69.0,NaN
3,1991-339,Adelaide,Hard,32.0,A,19901231,4,101889,8.0,NaN,Todd Woodbridge,R,178.0,AUS,19.7,101703,NaN,NaN,Guillaume Raoux,R,180.0,FRA,20.8,7-6(2) 6-1,3,R32,85.0,2.0,0.0,60.0,40.0,30.0,14.0,9.0,3.0,3.0,3.0,3.0,74.0,45.0,30.0,11.0,10.0,5.0,8.0,50.0,NaN,84.0,NaN
4,1991-339,Adelaide,Hard,32.0,A,19901231,5,101274,NaN,NaN,Udo Riglewski,R,185.0,GER,24.4,101843,4.0,NaN,Sergi Bruguera,R,188.0,ESP,19.9,7-5 6-3,3,R32,90.0,4.0,2.0,72.0,40.0,33.0,14.0,10.0,7.0,8.0,2.0,2.0,77.0,41.0,28.0,15.0,11.0,4.0,8.0,88.0,NaN,28.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197258,2025-06-23_Eastbourne_2025-0741-381,Eastbourne,Grass,32.0,A,20250623,2025-0741-381,200221,NaN,NaN,Alejandro Davidovich Fokina,R,183.0,ESP,26.1,210150,NaN,NaN,Jakub Mensik,R,193.0,CZE,19.8,6-4 7-5,3,QF,104.0,1.0,4.0,76.0,46.0,33.0,17.0,11.0,6.0,7.0,9.0,4.0,85.0,55.0,35.0,14.0,11.0,7.0,10.0,28.0,NaN,17.0,NaN
197259,2025-06-23_Eastbourne_2025-0741-382,Eastbourne,Grass,32.0,A,20250623,2025-0741-382,126203,NaN,NaN,Taylor Fritz,R,196.0,USA,27.7,106218,NaN,NaN,Marcos Giron,R,180.0,USA,31.9,7-5 4-6 7-5,3,QF,144.0,16.0,1.0,99.0,72.0,58.0,10.0,17.0,1.0,4.0,3.0,5.0,102.0,56.0,42.0,22.0,17.0,1.0,5.0,5.0,NaN,46.0,NaN
197260,2025-06-23_Eastbourne_2025-0741-383,Eastbourne,Grass,32.0,A,20250623,2025-0741-383,202385,NaN,NaN,Jenson Brooksby,R,188.0,USA,24.7,200005,NaN,NaN,Ugo Humbert,L,188.0,FRA,27.0,6-7(7) 6-4 6-4,3,SF,167.0,4.0,6.0,104.0,56.0,39.0,26.0,16.0,9.0,13.0,4.0,3.0,103.0,65.0,44.0,16.0,16.0,7.0,13.0,149.0,NaN,20.0,NaN
197261,2025-06-23_Eastbourne_2025-0741-384,Eastbourne,Grass,32.0,A,20250623,2025-0741-384,126203,NaN,NaN,Taylor Fritz,R,196.0,USA,27.7,200221,NaN,NaN,Alejandro Davidovich Fokina,R,183.0,ESP,26.1,6-3 3-6 6-1,3,SF,93.0,18.0,0.0,77.0,54.0,42.0,12.0,13.0,2.0,3.0,1.0,2.0,65.0,43.0,30.0,11.0,12.0,0.0,3.0,5.0,NaN,28.0,NaN


Removing NaN values from the columns of interest did not remove that much data. We end up with a lot of data still, so this should be okay.

## Randomize Winner and Loser

The current dataset has labels like winner and loser. We want to randomize this and assign a Player 1 and Player 2. My first try at this project I did not do this. I think it's totally possible to not have any data leakage, and randomize it just before training (as I did in DataAnalysis.ipynb). However, there's less risk if we randomize from the beginning. So, let's do that.

These are the columns we should be switching:

In [5]:
columnsWinner = [
    "winner_id","winner_seed", "winner_entry", "winner_name", "winner_hand", "winner_ht", "winner_ioc", "winner_age", "w_ace", "w_df", "w_svpt", "w_1stIn", "w_1stWon", "w_2ndWon", "w_SvGms", "w_bpSaved", "w_bpFaced", "winner_rank","winner_rank_points"
]
columnsLoser = [
    "loser_id","loser_seed", "loser_entry", "loser_name", "loser_hand", "loser_ht", "loser_ioc", "loser_age", "l_ace", "l_df", "l_svpt", "l_1stIn", "l_1stWon", "l_2ndWon", "l_SvGms", "l_bpSaved", "l_bpFaced", "loser_rank","loser_rank_points"
]

Let's first rename them to Player 1 and Player 2.

In [6]:
# Creating a rename mapping
rename_dict = {col: col.replace("winner", "p1").replace("w_", "p1_") for col in columnsWinner}
rename_dict.update({col: col.replace("loser", "p2").replace("l_", "p2_") for col in columnsLoser})

# Renaming columns
all_data_filtered = all_data_filtered.rename(columns=rename_dict)
all_data_filtered

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,p1_id,p1_seed,p1_entry,p1_name,p1_hand,p1_ht,p1_ioc,p1_age,p2_id,p2_seed,p2_entry,p2_name,p2_hand,p2_ht,p2_ioc,p2_age,score,best_of,round,minutes,p1_ace,p1_df,p1_svpt,p1_1stIn,p1_1stWon,p1_2ndWon,p1_SvGms,p1_bpSaved,p1_bpFaced,p2_ace,p2_df,p2_svpt,p2_1stIn,p2_1stWon,p2_2ndWon,p2_SvGms,p2_bpSaved,p2_bpFaced,p1_rank,p1_rank_points,p2_rank,p2_rank_points
0,1991-339,Adelaide,Hard,32.0,A,19901231,1,101723,NaN,NaN,Magnus Larsson,R,193.0,SWE,20.7,101414,1.0,NaN,Boris Becker,R,190.0,GER,23.1,6-4 3-6 7-6(2),3,R32,130.0,6.0,2.0,96.0,55.0,39.0,25.0,15.0,2.0,4.0,8.0,3.0,95.0,62.0,44.0,23.0,16.0,6.0,8.0,56.0,NaN,2.0,NaN
1,1991-339,Adelaide,Hard,32.0,A,19901231,2,100946,NaN,Q,Slobodan Zivojinovic,R,198.0,YUG,27.4,101256,NaN,NaN,Mark Kratzmann,L,178.0,AUS,24.6,6-3 3-6 7-6(6),3,R32,119.0,19.0,4.0,101.0,56.0,45.0,25.0,15.0,9.0,10.0,8.0,2.0,84.0,41.0,35.0,27.0,15.0,1.0,2.0,304.0,NaN,75.0,NaN
2,1991-339,Adelaide,Hard,32.0,A,19901231,3,101234,NaN,NaN,Patrik Kuhnen,R,190.0,GER,24.8,101421,NaN,NaN,Veli Paloheimo,R,183.0,FIN,23.0,6-0 6-4,3,R32,71.0,6.0,1.0,54.0,31.0,24.0,13.0,8.0,1.0,1.0,2.0,2.0,60.0,37.0,22.0,6.0,8.0,4.0,8.0,82.0,NaN,69.0,NaN
3,1991-339,Adelaide,Hard,32.0,A,19901231,4,101889,8.0,NaN,Todd Woodbridge,R,178.0,AUS,19.7,101703,NaN,NaN,Guillaume Raoux,R,180.0,FRA,20.8,7-6(2) 6-1,3,R32,85.0,2.0,0.0,60.0,40.0,30.0,14.0,9.0,3.0,3.0,3.0,3.0,74.0,45.0,30.0,11.0,10.0,5.0,8.0,50.0,NaN,84.0,NaN
4,1991-339,Adelaide,Hard,32.0,A,19901231,5,101274,NaN,NaN,Udo Riglewski,R,185.0,GER,24.4,101843,4.0,NaN,Sergi Bruguera,R,188.0,ESP,19.9,7-5 6-3,3,R32,90.0,4.0,2.0,72.0,40.0,33.0,14.0,10.0,7.0,8.0,2.0,2.0,77.0,41.0,28.0,15.0,11.0,4.0,8.0,88.0,NaN,28.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197258,2025-06-23_Eastbourne_2025-0741-381,Eastbourne,Grass,32.0,A,20250623,2025-0741-381,200221,NaN,NaN,Alejandro Davidovich Fokina,R,183.0,ESP,26.1,210150,NaN,NaN,Jakub Mensik,R,193.0,CZE,19.8,6-4 7-5,3,QF,104.0,1.0,4.0,76.0,46.0,33.0,17.0,11.0,6.0,7.0,9.0,4.0,85.0,55.0,35.0,14.0,11.0,7.0,10.0,28.0,NaN,17.0,NaN
197259,2025-06-23_Eastbourne_2025-0741-382,Eastbourne,Grass,32.0,A,20250623,2025-0741-382,126203,NaN,NaN,Taylor Fritz,R,196.0,USA,27.7,106218,NaN,NaN,Marcos Giron,R,180.0,USA,31.9,7-5 4-6 7-5,3,QF,144.0,16.0,1.0,99.0,72.0,58.0,10.0,17.0,1.0,4.0,3.0,5.0,102.0,56.0,42.0,22.0,17.0,1.0,5.0,5.0,NaN,46.0,NaN
197260,2025-06-23_Eastbourne_2025-0741-383,Eastbourne,Grass,32.0,A,20250623,2025-0741-383,202385,NaN,NaN,Jenson Brooksby,R,188.0,USA,24.7,200005,NaN,NaN,Ugo Humbert,L,188.0,FRA,27.0,6-7(7) 6-4 6-4,3,SF,167.0,4.0,6.0,104.0,56.0,39.0,26.0,16.0,9.0,13.0,4.0,3.0,103.0,65.0,44.0,16.0,16.0,7.0,13.0,149.0,NaN,20.0,NaN
197261,2025-06-23_Eastbourne_2025-0741-384,Eastbourne,Grass,32.0,A,20250623,2025-0741-384,126203,NaN,NaN,Taylor Fritz,R,196.0,USA,27.7,200221,NaN,NaN,Alejandro Davidovich Fokina,R,183.0,ESP,26.1,6-3 3-6 6-1,3,SF,93.0,18.0,0.0,77.0,54.0,42.0,12.0,13.0,2.0,3.0,1.0,2.0,65.0,43.0,30.0,11.0,12.0,0.0,3.0,5.0,NaN,28.0,NaN


However, all the winners are now in P1. We don't want that, we want to switch them around 50% of the time.

In [7]:
# Generate a boolean mask for 50% of the rows (True means swap)
mask = np.random.rand(len(all_data_filtered)) < 0.5 

# Identify the player 1 and player 2 columns
player1_cols = [col for col in all_data_filtered.columns if "player1" in col or "p1_" in col]
player2_cols = [col for col in all_data_filtered.columns if "player2" in col or "p2_" in col]

# Create the RESULT column (1 = not swapped, 0 = swapped)
all_data_filtered["RESULT"] = np.where(mask, 0, 1)

# Swap values where mask is True
all_data_filtered.loc[mask, player1_cols], all_data_filtered.loc[mask, player2_cols] = all_data_filtered.loc[mask, player2_cols].values, all_data_filtered.loc[mask, player1_cols].values

In [8]:
all_data_filtered

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,p1_id,p1_seed,p1_entry,p1_name,p1_hand,p1_ht,p1_ioc,p1_age,p2_id,p2_seed,p2_entry,p2_name,p2_hand,p2_ht,p2_ioc,p2_age,score,best_of,round,minutes,p1_ace,p1_df,p1_svpt,p1_1stIn,p1_1stWon,p1_2ndWon,p1_SvGms,p1_bpSaved,p1_bpFaced,p2_ace,p2_df,p2_svpt,p2_1stIn,p2_1stWon,p2_2ndWon,p2_SvGms,p2_bpSaved,p2_bpFaced,p1_rank,p1_rank_points,p2_rank,p2_rank_points,RESULT
0,1991-339,Adelaide,Hard,32.0,A,19901231,1,101723,NaN,NaN,Magnus Larsson,R,193.0,SWE,20.7,101414,1.0,NaN,Boris Becker,R,190.0,GER,23.1,6-4 3-6 7-6(2),3,R32,130.0,6.0,2.0,96.0,55.0,39.0,25.0,15.0,2.0,4.0,8.0,3.0,95.0,62.0,44.0,23.0,16.0,6.0,8.0,56.0,NaN,2.0,NaN,1
1,1991-339,Adelaide,Hard,32.0,A,19901231,2,101256,NaN,NaN,Mark Kratzmann,L,178.0,AUS,24.6,100946,NaN,Q,Slobodan Zivojinovic,R,198.0,YUG,27.4,6-3 3-6 7-6(6),3,R32,119.0,8.0,2.0,84.0,41.0,35.0,27.0,15.0,1.0,2.0,19.0,4.0,101.0,56.0,45.0,25.0,15.0,9.0,10.0,75.0,NaN,304.0,NaN,0
2,1991-339,Adelaide,Hard,32.0,A,19901231,3,101234,NaN,NaN,Patrik Kuhnen,R,190.0,GER,24.8,101421,NaN,NaN,Veli Paloheimo,R,183.0,FIN,23.0,6-0 6-4,3,R32,71.0,6.0,1.0,54.0,31.0,24.0,13.0,8.0,1.0,1.0,2.0,2.0,60.0,37.0,22.0,6.0,8.0,4.0,8.0,82.0,NaN,69.0,NaN,1
3,1991-339,Adelaide,Hard,32.0,A,19901231,4,101703,NaN,NaN,Guillaume Raoux,R,180.0,FRA,20.8,101889,8.0,NaN,Todd Woodbridge,R,178.0,AUS,19.7,7-6(2) 6-1,3,R32,85.0,3.0,3.0,74.0,45.0,30.0,11.0,10.0,5.0,8.0,2.0,0.0,60.0,40.0,30.0,14.0,9.0,3.0,3.0,84.0,NaN,50.0,NaN,0
4,1991-339,Adelaide,Hard,32.0,A,19901231,5,101274,NaN,NaN,Udo Riglewski,R,185.0,GER,24.4,101843,4.0,NaN,Sergi Bruguera,R,188.0,ESP,19.9,7-5 6-3,3,R32,90.0,4.0,2.0,72.0,40.0,33.0,14.0,10.0,7.0,8.0,2.0,2.0,77.0,41.0,28.0,15.0,11.0,4.0,8.0,88.0,NaN,28.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197258,2025-06-23_Eastbourne_2025-0741-381,Eastbourne,Grass,32.0,A,20250623,2025-0741-381,200221,NaN,NaN,Alejandro Davidovich Fokina,R,183.0,ESP,26.1,210150,NaN,NaN,Jakub Mensik,R,193.0,CZE,19.8,6-4 7-5,3,QF,104.0,1.0,4.0,76.0,46.0,33.0,17.0,11.0,6.0,7.0,9.0,4.0,85.0,55.0,35.0,14.0,11.0,7.0,10.0,28.0,NaN,17.0,NaN,1
197259,2025-06-23_Eastbourne_2025-0741-382,Eastbourne,Grass,32.0,A,20250623,2025-0741-382,106218,NaN,NaN,Marcos Giron,R,180.0,USA,31.9,126203,NaN,NaN,Taylor Fritz,R,196.0,USA,27.7,7-5 4-6 7-5,3,QF,144.0,3.0,5.0,102.0,56.0,42.0,22.0,17.0,1.0,5.0,16.0,1.0,99.0,72.0,58.0,10.0,17.0,1.0,4.0,46.0,NaN,5.0,NaN,0
197260,2025-06-23_Eastbourne_2025-0741-383,Eastbourne,Grass,32.0,A,20250623,2025-0741-383,200005,NaN,NaN,Ugo Humbert,L,188.0,FRA,27.0,202385,NaN,NaN,Jenson Brooksby,R,188.0,USA,24.7,6-7(7) 6-4 6-4,3,SF,167.0,4.0,3.0,103.0,65.0,44.0,16.0,16.0,7.0,13.0,4.0,6.0,104.0,56.0,39.0,26.0,16.0,9.0,13.0,20.0,NaN,149.0,NaN,0
197261,2025-06-23_Eastbourne_2025-0741-384,Eastbourne,Grass,32.0,A,20250623,2025-0741-384,126203,NaN,NaN,Taylor Fritz,R,196.0,USA,27.7,200221,NaN,NaN,Alejandro Davidovich Fokina,R,183.0,ESP,26.1,6-3 3-6 6-1,3,SF,93.0,18.0,0.0,77.0,54.0,42.0,12.0,13.0,2.0,3.0,1.0,2.0,65.0,43.0,30.0,11.0,12.0,0.0,3.0,5.0,NaN,28.0,NaN,1


Let's drop the ATP rank points as it has changed accross the years.

In [9]:
all_data_filtered = all_data_filtered.drop('p2_rank_points', axis=1)
all_data_filtered = all_data_filtered.drop('p1_rank_points', axis=1)
all_data_filtered

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,p1_id,p1_seed,p1_entry,p1_name,p1_hand,p1_ht,p1_ioc,p1_age,p2_id,p2_seed,p2_entry,p2_name,p2_hand,p2_ht,p2_ioc,p2_age,score,best_of,round,minutes,p1_ace,p1_df,p1_svpt,p1_1stIn,p1_1stWon,p1_2ndWon,p1_SvGms,p1_bpSaved,p1_bpFaced,p2_ace,p2_df,p2_svpt,p2_1stIn,p2_1stWon,p2_2ndWon,p2_SvGms,p2_bpSaved,p2_bpFaced,p1_rank,p2_rank,RESULT
0,1991-339,Adelaide,Hard,32.0,A,19901231,1,101723,NaN,NaN,Magnus Larsson,R,193.0,SWE,20.7,101414,1.0,NaN,Boris Becker,R,190.0,GER,23.1,6-4 3-6 7-6(2),3,R32,130.0,6.0,2.0,96.0,55.0,39.0,25.0,15.0,2.0,4.0,8.0,3.0,95.0,62.0,44.0,23.0,16.0,6.0,8.0,56.0,2.0,1
1,1991-339,Adelaide,Hard,32.0,A,19901231,2,101256,NaN,NaN,Mark Kratzmann,L,178.0,AUS,24.6,100946,NaN,Q,Slobodan Zivojinovic,R,198.0,YUG,27.4,6-3 3-6 7-6(6),3,R32,119.0,8.0,2.0,84.0,41.0,35.0,27.0,15.0,1.0,2.0,19.0,4.0,101.0,56.0,45.0,25.0,15.0,9.0,10.0,75.0,304.0,0
2,1991-339,Adelaide,Hard,32.0,A,19901231,3,101234,NaN,NaN,Patrik Kuhnen,R,190.0,GER,24.8,101421,NaN,NaN,Veli Paloheimo,R,183.0,FIN,23.0,6-0 6-4,3,R32,71.0,6.0,1.0,54.0,31.0,24.0,13.0,8.0,1.0,1.0,2.0,2.0,60.0,37.0,22.0,6.0,8.0,4.0,8.0,82.0,69.0,1
3,1991-339,Adelaide,Hard,32.0,A,19901231,4,101703,NaN,NaN,Guillaume Raoux,R,180.0,FRA,20.8,101889,8.0,NaN,Todd Woodbridge,R,178.0,AUS,19.7,7-6(2) 6-1,3,R32,85.0,3.0,3.0,74.0,45.0,30.0,11.0,10.0,5.0,8.0,2.0,0.0,60.0,40.0,30.0,14.0,9.0,3.0,3.0,84.0,50.0,0
4,1991-339,Adelaide,Hard,32.0,A,19901231,5,101274,NaN,NaN,Udo Riglewski,R,185.0,GER,24.4,101843,4.0,NaN,Sergi Bruguera,R,188.0,ESP,19.9,7-5 6-3,3,R32,90.0,4.0,2.0,72.0,40.0,33.0,14.0,10.0,7.0,8.0,2.0,2.0,77.0,41.0,28.0,15.0,11.0,4.0,8.0,88.0,28.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197258,2025-06-23_Eastbourne_2025-0741-381,Eastbourne,Grass,32.0,A,20250623,2025-0741-381,200221,NaN,NaN,Alejandro Davidovich Fokina,R,183.0,ESP,26.1,210150,NaN,NaN,Jakub Mensik,R,193.0,CZE,19.8,6-4 7-5,3,QF,104.0,1.0,4.0,76.0,46.0,33.0,17.0,11.0,6.0,7.0,9.0,4.0,85.0,55.0,35.0,14.0,11.0,7.0,10.0,28.0,17.0,1
197259,2025-06-23_Eastbourne_2025-0741-382,Eastbourne,Grass,32.0,A,20250623,2025-0741-382,106218,NaN,NaN,Marcos Giron,R,180.0,USA,31.9,126203,NaN,NaN,Taylor Fritz,R,196.0,USA,27.7,7-5 4-6 7-5,3,QF,144.0,3.0,5.0,102.0,56.0,42.0,22.0,17.0,1.0,5.0,16.0,1.0,99.0,72.0,58.0,10.0,17.0,1.0,4.0,46.0,5.0,0
197260,2025-06-23_Eastbourne_2025-0741-383,Eastbourne,Grass,32.0,A,20250623,2025-0741-383,200005,NaN,NaN,Ugo Humbert,L,188.0,FRA,27.0,202385,NaN,NaN,Jenson Brooksby,R,188.0,USA,24.7,6-7(7) 6-4 6-4,3,SF,167.0,4.0,3.0,103.0,65.0,44.0,16.0,16.0,7.0,13.0,4.0,6.0,104.0,56.0,39.0,26.0,16.0,9.0,13.0,20.0,149.0,0
197261,2025-06-23_Eastbourne_2025-0741-384,Eastbourne,Grass,32.0,A,20250623,2025-0741-384,126203,NaN,NaN,Taylor Fritz,R,196.0,USA,27.7,200221,NaN,NaN,Alejandro Davidovich Fokina,R,183.0,ESP,26.1,6-3 3-6 6-1,3,SF,93.0,18.0,0.0,77.0,54.0,42.0,12.0,13.0,2.0,3.0,1.0,2.0,65.0,43.0,30.0,11.0,12.0,0.0,3.0,5.0,28.0,1


In [10]:
all_data_filtered['tourney_date'] = all_data_filtered['tourney_date'].astype(int)
all_data_filtered

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,p1_id,p1_seed,p1_entry,p1_name,p1_hand,p1_ht,p1_ioc,p1_age,p2_id,p2_seed,p2_entry,p2_name,p2_hand,p2_ht,p2_ioc,p2_age,score,best_of,round,minutes,p1_ace,p1_df,p1_svpt,p1_1stIn,p1_1stWon,p1_2ndWon,p1_SvGms,p1_bpSaved,p1_bpFaced,p2_ace,p2_df,p2_svpt,p2_1stIn,p2_1stWon,p2_2ndWon,p2_SvGms,p2_bpSaved,p2_bpFaced,p1_rank,p2_rank,RESULT
0,1991-339,Adelaide,Hard,32.0,A,19901231,1,101723,NaN,NaN,Magnus Larsson,R,193.0,SWE,20.7,101414,1.0,NaN,Boris Becker,R,190.0,GER,23.1,6-4 3-6 7-6(2),3,R32,130.0,6.0,2.0,96.0,55.0,39.0,25.0,15.0,2.0,4.0,8.0,3.0,95.0,62.0,44.0,23.0,16.0,6.0,8.0,56.0,2.0,1
1,1991-339,Adelaide,Hard,32.0,A,19901231,2,101256,NaN,NaN,Mark Kratzmann,L,178.0,AUS,24.6,100946,NaN,Q,Slobodan Zivojinovic,R,198.0,YUG,27.4,6-3 3-6 7-6(6),3,R32,119.0,8.0,2.0,84.0,41.0,35.0,27.0,15.0,1.0,2.0,19.0,4.0,101.0,56.0,45.0,25.0,15.0,9.0,10.0,75.0,304.0,0
2,1991-339,Adelaide,Hard,32.0,A,19901231,3,101234,NaN,NaN,Patrik Kuhnen,R,190.0,GER,24.8,101421,NaN,NaN,Veli Paloheimo,R,183.0,FIN,23.0,6-0 6-4,3,R32,71.0,6.0,1.0,54.0,31.0,24.0,13.0,8.0,1.0,1.0,2.0,2.0,60.0,37.0,22.0,6.0,8.0,4.0,8.0,82.0,69.0,1
3,1991-339,Adelaide,Hard,32.0,A,19901231,4,101703,NaN,NaN,Guillaume Raoux,R,180.0,FRA,20.8,101889,8.0,NaN,Todd Woodbridge,R,178.0,AUS,19.7,7-6(2) 6-1,3,R32,85.0,3.0,3.0,74.0,45.0,30.0,11.0,10.0,5.0,8.0,2.0,0.0,60.0,40.0,30.0,14.0,9.0,3.0,3.0,84.0,50.0,0
4,1991-339,Adelaide,Hard,32.0,A,19901231,5,101274,NaN,NaN,Udo Riglewski,R,185.0,GER,24.4,101843,4.0,NaN,Sergi Bruguera,R,188.0,ESP,19.9,7-5 6-3,3,R32,90.0,4.0,2.0,72.0,40.0,33.0,14.0,10.0,7.0,8.0,2.0,2.0,77.0,41.0,28.0,15.0,11.0,4.0,8.0,88.0,28.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197258,2025-06-23_Eastbourne_2025-0741-381,Eastbourne,Grass,32.0,A,20250623,2025-0741-381,200221,NaN,NaN,Alejandro Davidovich Fokina,R,183.0,ESP,26.1,210150,NaN,NaN,Jakub Mensik,R,193.0,CZE,19.8,6-4 7-5,3,QF,104.0,1.0,4.0,76.0,46.0,33.0,17.0,11.0,6.0,7.0,9.0,4.0,85.0,55.0,35.0,14.0,11.0,7.0,10.0,28.0,17.0,1
197259,2025-06-23_Eastbourne_2025-0741-382,Eastbourne,Grass,32.0,A,20250623,2025-0741-382,106218,NaN,NaN,Marcos Giron,R,180.0,USA,31.9,126203,NaN,NaN,Taylor Fritz,R,196.0,USA,27.7,7-5 4-6 7-5,3,QF,144.0,3.0,5.0,102.0,56.0,42.0,22.0,17.0,1.0,5.0,16.0,1.0,99.0,72.0,58.0,10.0,17.0,1.0,4.0,46.0,5.0,0
197260,2025-06-23_Eastbourne_2025-0741-383,Eastbourne,Grass,32.0,A,20250623,2025-0741-383,200005,NaN,NaN,Ugo Humbert,L,188.0,FRA,27.0,202385,NaN,NaN,Jenson Brooksby,R,188.0,USA,24.7,6-7(7) 6-4 6-4,3,SF,167.0,4.0,3.0,103.0,65.0,44.0,16.0,16.0,7.0,13.0,4.0,6.0,104.0,56.0,39.0,26.0,16.0,9.0,13.0,20.0,149.0,0
197261,2025-06-23_Eastbourne_2025-0741-384,Eastbourne,Grass,32.0,A,20250623,2025-0741-384,126203,NaN,NaN,Taylor Fritz,R,196.0,USA,27.7,200221,NaN,NaN,Alejandro Davidovich Fokina,R,183.0,ESP,26.1,6-3 3-6 6-1,3,SF,93.0,18.0,0.0,77.0,54.0,42.0,12.0,13.0,2.0,3.0,1.0,2.0,65.0,43.0,30.0,11.0,12.0,0.0,3.0,5.0,28.0,1


In [11]:
all_data_filtered = all_data_filtered.sort_values(["tourney_date", "tourney_name"])
all_data_filtered

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,p1_id,p1_seed,p1_entry,p1_name,p1_hand,p1_ht,p1_ioc,p1_age,p2_id,p2_seed,p2_entry,p2_name,p2_hand,p2_ht,p2_ioc,p2_age,score,best_of,round,minutes,p1_ace,p1_df,p1_svpt,p1_1stIn,p1_1stWon,p1_2ndWon,p1_SvGms,p1_bpSaved,p1_bpFaced,p2_ace,p2_df,p2_svpt,p2_1stIn,p2_1stWon,p2_2ndWon,p2_SvGms,p2_bpSaved,p2_bpFaced,p1_rank,p2_rank,RESULT
0,1991-339,Adelaide,Hard,32.0,A,19901231,1,101723,NaN,NaN,Magnus Larsson,R,193.0,SWE,20.7,101414,1.0,NaN,Boris Becker,R,190.0,GER,23.1,6-4 3-6 7-6(2),3,R32,130.0,6.0,2.0,96.0,55.0,39.0,25.0,15.0,2.0,4.0,8.0,3.0,95.0,62.0,44.0,23.0,16.0,6.0,8.0,56.0,2.0,1
1,1991-339,Adelaide,Hard,32.0,A,19901231,2,101256,NaN,NaN,Mark Kratzmann,L,178.0,AUS,24.6,100946,NaN,Q,Slobodan Zivojinovic,R,198.0,YUG,27.4,6-3 3-6 7-6(6),3,R32,119.0,8.0,2.0,84.0,41.0,35.0,27.0,15.0,1.0,2.0,19.0,4.0,101.0,56.0,45.0,25.0,15.0,9.0,10.0,75.0,304.0,0
2,1991-339,Adelaide,Hard,32.0,A,19901231,3,101234,NaN,NaN,Patrik Kuhnen,R,190.0,GER,24.8,101421,NaN,NaN,Veli Paloheimo,R,183.0,FIN,23.0,6-0 6-4,3,R32,71.0,6.0,1.0,54.0,31.0,24.0,13.0,8.0,1.0,1.0,2.0,2.0,60.0,37.0,22.0,6.0,8.0,4.0,8.0,82.0,69.0,1
3,1991-339,Adelaide,Hard,32.0,A,19901231,4,101703,NaN,NaN,Guillaume Raoux,R,180.0,FRA,20.8,101889,8.0,NaN,Todd Woodbridge,R,178.0,AUS,19.7,7-6(2) 6-1,3,R32,85.0,3.0,3.0,74.0,45.0,30.0,11.0,10.0,5.0,8.0,2.0,0.0,60.0,40.0,30.0,14.0,9.0,3.0,3.0,84.0,50.0,0
4,1991-339,Adelaide,Hard,32.0,A,19901231,5,101274,NaN,NaN,Udo Riglewski,R,185.0,GER,24.4,101843,4.0,NaN,Sergi Bruguera,R,188.0,ESP,19.9,7-5 6-3,3,R32,90.0,4.0,2.0,72.0,40.0,33.0,14.0,10.0,7.0,8.0,2.0,2.0,77.0,41.0,28.0,15.0,11.0,4.0,8.0,88.0,28.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195110,2025-06-23_Milan CH_2025-3463-377,Milan CH,Clay,32.0,C,20250623,2025-3463-377,122352,NaN,NaN,Christoph Negritu,R,193.0,GER,31.3,123809,NaN,NaN,Ivan Gakhov,L,191.0,RUS,28.6,6-3 6-4,3,R16,98.0,6.0,2.0,85.0,54.0,40.0,12.0,10.0,9.0,10.0,4.0,6.0,54.0,32.0,24.0,6.0,9.0,2.0,5.0,290.0,270.0,1
195111,2025-06-23_Milan CH_2025-3463-378,Milan CH,Clay,32.0,C,20250623,2025-3463-378,209976,NaN,NaN,Dino Prizmic,R,188.0,CRO,19.9,202235,NaN,NaN,Jacopo Berrettini,R,193.0,ITA,26.6,6-1 6-2,3,R16,84.0,2.0,5.0,46.0,22.0,16.0,13.0,8.0,3.0,5.0,0.0,3.0,41.0,20.0,7.0,8.0,7.0,1.0,7.0,173.0,330.0,1
195112,2025-06-23_Milan CH_2025-3463-380,Milan CH,Clay,32.0,C,20250623,2025-3463-380,106065,NaN,NaN,Marco Cecchinato,R,185.0,ITA,32.7,208069,NaN,NaN,Max Houkes,L,188.0,NED,25.0,6-4 6-2,3,QF,92.0,1.0,3.0,45.0,30.0,26.0,11.0,9.0,2.0,2.0,3.0,6.0,63.0,35.0,22.0,13.0,9.0,2.0,5.0,437.0,256.0,1
195113,2025-06-23_Milan CH_2025-3463-382,Milan CH,Clay,32.0,C,20250623,2025-3463-382,122352,NaN,NaN,Christoph Negritu,R,193.0,GER,31.3,209976,NaN,NaN,Dino Prizmic,R,188.0,CRO,19.9,6-4 6-3,3,QF,94.0,0.0,0.0,45.0,29.0,15.0,6.0,9.0,1.0,6.0,1.0,6.0,64.0,28.0,20.0,17.0,10.0,3.0,6.0,290.0,173.0,0


And volià! Now we have a clean dataset with Player 1 and Player 2 randomized.

**RESULT 1 - Means that Player 1 WON**

**RESULT 0 - Means that Player 2 WON**

Let's export this to "./data/0cleanDataset.csv"

In [12]:
# all_data_filtered.to_csv("./data/0cleanDatasetWithQualifiersWith2025.csv", index=False)